In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset

In [ ]:
# Cargar el conjunto de datos
data = pd.read_csv('/content/MNIST Digits.csv')

In [ ]:
# Dividir los datos en características (X) y etiquetas (y)
X = data.drop('label', axis=1).values
y = data['label'].values

# Codificar las etiquetas en valores numéricos
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Definir una clase personalizada para el conjunto de datos
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Crear instancias del conjunto de datos
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
# Definir el modelo del perceptrón
class Perceptron(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = self.fc(x)
        return x

# Definir los parámetros del modelo
input_size = X.shape[1]
num_classes = len(label_encoder.classes_)

# Crear una instancia del modelo
model = Perceptron(input_size, num_classes)

In [ ]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Definir los dataloaders para el entrenamiento y la prueba
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
# Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 454.2859
Epoch [2/10], Loss: 73.0228
Epoch [3/10], Loss: 112.7605
Epoch [4/10], Loss: 184.6001
Epoch [5/10], Loss: 114.5045
Epoch [6/10], Loss: 279.4065
Epoch [7/10], Loss: 0.0000
Epoch [8/10], Loss: 152.2698
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 162.8566


In [ ]:
# Evaluación del modelo
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8679
